In [ ]:
from cdcqr.data.glassnode.utils import get_glassnode_data, quantile_info, url_parser
from datetime import datetime as dt
from datetime import timedelta
from cdcqr.common.config import CDQ_REPO_DIR
from cdcqr.common.utils import camel_case2snake_case

import sys
sys.path.append(CDQ_REPO_DIR)
from ct.utils import glassnode
from cdcqr.common.config import GLASSNODE_API_KEY
GLASSKEY = GLASSNODE_API_KEY
import requests
import pandas as pd
from cdcqr.common.utils import load_df


In [ ]:
category = 'Volume'
volume_uxto_volume = ['UtxoCreatedValueSum', 
                    'UtxoSpentValueMean', 
                     ]

In [ ]:
category='transactions'
feature='TransfersVolumeToExchangesSum'
underly = 'BTC'
freq='1h'

start_time = dt.now()-timedelta(hours=8760)
u_time = dt.now()-timedelta(hours=1)
url1 = 'metrics~{}~{}'.format(category, camel_case2snake_case(feature))
print(url1, underly, freq, start_time)

dff = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
dff

In [ ]:
#category='volume'
#feature='UtxoCreatedValueSum'

category='transactions'
category='mining'
category = 'addresses'
feature='TransfersVolumeToExchangesSum'
feature='ActiveCount'

underly = 'BTC'
freq='1h'

start_time = dt.now()-timedelta(hours=48)
u_time = dt.now()-timedelta(hours=1)
url1 = 'metrics~{}~{}'.format(category, camel_case2snake_case(feature))
print(url1, underly, freq, start_time)

dff = glassnode(url1,a=underly,i=freq,s=start_time,price=False)

In [ ]:
catplusf = 'derivatives.FuturesFundingRatePerpetual'

category, feature = catplusf.split('.')

underly = 'BTC'
freq='10m'

start_time = dt.now()-timedelta(hours=9000)
u_time = dt.now()-timedelta(hours=1)
url1 = 'metrics~{}~{}'.format(category, camel_case2snake_case(feature))
print(url1, underly, freq, start_time)

dff = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
dff

In [ ]:
dfbtc = load_df('BTCUSD_PERP@binance').resample_pv('10Min')
dfeth = load_df('ETHUSD_PERP@binance').resample_pv('10Min')
df_underly = {}
df_underly['BTC'] = dfbtc
df_underly['ETH'] = dfeth

In [ ]:
import cdcqr.analytics.utils as autil 
%matplotlib inline


In [ ]:
freq = '10m'

In [ ]:
category = 'derivatives'
features = ['FuturesFundingRatePerpetual']
for feature_name in features:
    for underly in ['BTC','ETH']:
        
        start_time = dt.now()-timedelta(hours=9000)
        url1 = 'metrics~{}~{}'.format(category, camel_case2snake_case(feature_name))
        print(url1, underly, freq, start_time)
        dff = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        print(dff.head())
        if True:
            common_idx = dff.index.intersection(dfbtc.index)
            dff = dff.reindex(index = common_idx)
            dfy = df_underly[underly]
            dfy = dfy.reindex(index = common_idx)
            dff[underly] = dfy['c']
            dff[feature_name] = dff['v']
            dff = dff[[feature_name,underly]]

            dff['f_n1'] = dff[feature_name]
            _ = autil.ts_eda(dff, feature_name, 'f_n1', underly)

In [ ]:
import ct.mlexp2 as ml

df_dict = df_underly

In [ ]:
category = 'derivatives'
feature = 'FuturesFundingRatePerpetual'

for underly in ['BTC','ETH']:
    df_i = df_dict[underly]
    dfgsi = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
    df_i = df_i[df_i.index<='20211031']
    dff = dfgsi.reindex(df_i.index)
    #dff['v'] = dff[feature]
    display(dff.head())
    res = ml.mlexp2(dff, df_i, sym=underly,
          qsuffix='USDT@binancealt',
          ytrain_thresh=6,
          ytrain_periods=100,
          ytest_thresh=3,
          ytest_periods=100,
          vdelay=2,
          ytypes=["ys"],
          xtypes=["v"],
          qlags=1000,
          request=['vbt','vbtplot'],
          #models=["oneiflarger","dt"],
          models=["oneiflarger", 'dt'],
          randcv_n_iter=30,
          sl=2,
          tp=6,
          ts=None,
          vbtfreq="10Min") 